<h1>
<center>
Module 3: Gini assignment
</center>
</h1>
<div class=h1_cell>

You will be working with the loan table again.

</div>

In [1]:
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/399/notebook_history/loan_wrangled_week2.csv', 'r') as f:
  loan_table = pd.read_csv(f)

We changed the library so need to update it for this notebook.

In [4]:
from google.colab import files
files.upload()

Saving library_w19_week3.py to library_w19_week3.py


{'library_w19_week3.py': b"def predictor_case(row, pred, target):\r\n  case_dict = {(0,0): 'true_negative', (1,1): 'true_positive', (0,1): 'false_negative', (1,0): 'false_positive'}\r\n  actual = row[target]\r\n  prediction = row[pred]\r\n  case = case_dict[(prediction, actual)]\r\n  return case\r\n\r\ndef accuracy(cases):\r\n    tp = cases['true_positive']\r\n    tn = cases['true_negative']\r\n    fp = cases['false_positive']\r\n    fn = cases['false_negative']\r\n    return (tp + tn)/(tp+tn+fp+fn)\r\n\r\ndef f1(cases):\r\n    #the heart of the matrix\r\n    tp = cases['true_positive']\r\n    fn = cases['false_negative']\r\n    tn = cases['true_negative']\r\n    fp = cases['false_positive']\r\n    \r\n    #other measures we can derive\r\n    recall = 1.0*tp/(tp+fn)  # positive correct divided by total positive in the table\r\n    precision = 1.0*tp/(tp+fp) # positive correct divided by all positive predictions made\r\n    \r\n    #now for the one we want\r\n    f1 = 2/(1/recall + 1/pr

In [5]:
#load the lbirary for this week to use what was defined in content notebook

from library_w19_week3 import *
%who function

accuracy	 f1	 gig	 gini	 informedness	 predictor_case	 probabilities	 


In [6]:
pd.set_option('display.max_columns', None)
loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0


<hr>
<h1>
1: One hot encode ApplicantIncome
</h1>
<p>
<div class=h1_cell>
First bin it and then one hot encode the binned values. Use ['low', 'average', 'high'] as the 3 binned values.
</div>

In [7]:
#bin ApplicantIncome
bins = 3
bin_names = ['low', 'average', 'high']
loan_table['apin_binned'] = pd.cut(loan_table['ApplicantIncome'], bins, labels=bin_names)

loan_table.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,0,0,128.000000,1,0,0,0,Low,1,0,0,0,1,0,low
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,1,0,66.000000,0,0,1,0,Low,1,0,0,0,1,0,low
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,1,0,120.000000,0,0,1,0,Low,1,0,0,0,1,0,low
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,1,0,141.000000,0,0,1,0,Low,1,0,0,0,1,0,low


In [11]:
#ohe apin_binned
one_hot_apin = pd.get_dummies(loan_table['apin_binned'], prefix='apin', dummy_na=True)
loan_table = loan_table.join(one_hot_apin)

loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned,apin_low,apin_average,apin_high,apin_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0


<hr>
<h1>
2: Use `ch_bad` as root splitter
</h1>
<p>
<div class=h1_cell>
Build the 2 sub-tables from splitting with `ch_bad` on the root node.
</div>

In [12]:
true_bad = loan_table.loc[loan_table['ch_bad'] == 1]
false_bad = loan_table.loc[loan_table['ch_bad'] == 0]

print((len(true_bad), len(false_bad)))

(89, 525)


<hr>
<h1>
3: Determine the best splitter for the true_bad table
</h1>
<p>
<div class=h1_cell>
Using the `true_bad` table as the root, compute a sorted list of the `apin_*` columns based on their gig scores.
</div>

In [26]:
columns = ['apin_average', 'apin_low', 'apin_high', 'apin_nan']

gig_scores = map(lambda col: (col, gig(true_bad, col, 'Loan_Status')), columns)
gig_sorted = sorted(gig_scores, key=lambda item: item[1], reverse=True)

gig_sorted

[('apin_average', 0.019292789018833467),
 ('apin_low', 0.008162501251583493),
 ('apin_high', 0.00014059290034529326),
 ('apin_nan', 0.0)]

<hr>
<h1>
4: Explore Entropy
</h1>
<p>
<div class=h1_cell>
<p>
Entropy is an alternative measure to Gini. Here is the formula you need, where p(a) is probablity of Loan_Status being 1 and p(b) is probability of Loan_Status being 0. Also, log is in base 2.
<p>
`Entropy = - p(a)*log(p(a)) - p(b)*log(p(b))`
<p>
Go ahead and define a function `entropy` that takes in a Series object with counts (which I'll give you). It returns the entropy score as above.
<p>
Hint: the `apin_nan` splitter will give you grief if you are not careful. It will produce an empty table on one branch. Be careful to avoid dividing by zero or taking the log of 0.
</div>

In [28]:
#first get probabilities for entire table
root_counts = loan_table['Loan_Status'].value_counts()

root_counts  #422 successful applications

1    422
0    192
Name: Loan_Status, dtype: int64

In [0]:
# watch out for 0 valued probs

import math

def entropy(counts):
  (p0, p1) = probabilities(counts)
  left = 0 if p0 == 0 else -(p0 * math.log(p0, 2))
  right = 0 if p1 == 0 else (p1 * math.log(p1, 2))
  return left - right

In [37]:
#compute entropy of entire loan_table before any splits

entropy(root_counts)

0.8962696698442932

<div class=just_text>
Like Gini, to use Entropy for a split on column C, compute the Entropy of the 3 tables, i.e., the starting table, the true table (column C is true) and the false table (column C is false). We now have 3 separate Entropy values for the 3 tables involved. Our next step is to combine these values in a way  that gives us an overall goodness-of-the-split score. To do so, we will use something called *`information gain`* (IGain). The formula for IGain is as follows:
<p>
`IGain = start_entropy − (w_true * entropy_true + w_false * entropy_false)`
<p>
where w_true is a weight of |true_table|/|titanic_table| (i.e., the size of true_table divided by the size of that starting table) and w_false is a weight of |false_table|/|titanic_table|.
<p>
Notice anything? Igain is equivalent to GIG at this point. It combines its 3 scores in the same way. It is just the indiviual scoring method that differs.
<p>
Please define an IGain function and use it to calculate a split on loan_table. Compare it with the gig score on the same split.
</div>

In [0]:
def IGain(starting_table, split_column, target_column):
  true_table = starting_table.loc[starting_table[split_column] == 1]
  false_table = starting_table.loc[starting_table[split_column] == 0]

  true_counts = true_table[target_column].value_counts()
  false_counts = false_table[target_column].value_counts()
  starting_counts = starting_table[target_column].value_counts() 

  #compute the entropy for the 3 tables
  starting_entropy = entropy(starting_counts)
  true_entropy = entropy(true_counts)
  false_entropy = entropy(false_counts)

  #compute the weights
  starting_size = len(starting_table.index)
  true_weight = 0.0 if starting_size == 0 else len(true_table.index)/starting_size
  false_weight = 0.0 if starting_size == 0 else len(false_table.index)/starting_size

  #wrap it up and put on a bow
  IGain = starting_entropy - (true_weight * true_entropy + false_weight * false_entropy)

  return IGain

In [43]:
#gig scores computed here so we can compare


gig_sorted

[('apin_average', 0.019292789018833467),
 ('apin_low', 0.008162501251583493),
 ('apin_high', 0.00014059290034529326),
 ('apin_nan', 0.0)]

In [44]:
#igain scores computed here
columns = ['apin_average', 'apin_low', 'apin_high', 'apin_nan']

igain_scores = map(lambda col: (col, IGain(loan_table, col, 'Loan_Status')), columns)
igain_sorted = sorted(igain_scores, key=lambda item: item[1], reverse=True)

igain_sorted

[('apin_average', 0.0003798431101625921),
 ('apin_high', 0.000356511111195279),
 ('apin_low', 2.8658495986100263e-05),
 ('apin_nan', 0.0)]

<div class=just_text>
There is a difference but I conjecture that the scores are so low that none of these look like good choices as a splitter. So the difference is in the noise. There are a lot of other columns to explore but I will hold off for now.
  <p>
    
</div>

<hr>
<h1>Write it out</h1>
<div class=h1_cell>

We made changes so save the table so can use it in next module.
</div>

In [45]:
loan_table.head(1)

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,no_lam,filled_lam,pa_Rural,pa_Semiurban,pa_Urban,pa_nan,lam_bin,lam_Low,lam_Average,lam_High,ch_bad,ch_good,ch_nan,apin_binned,apin_low,apin_average,apin_high,apin_nan
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,1,1,146.412162,0,0,1,0,Low,1,0,0,0,1,0,low,1,0,0,0


In [0]:
with open('/content/gdrive/My Drive/Colab Notebooks/399/notebook_history/loan_wrangled_week3.csv', 'w') as f:
  loan_table.to_csv(f, encoding='utf-8', index=False)

<hr>
<h1>No updates for library</h1>
<div class=h1_cell>

We defined new  entropy functions but we won't be using them later - we will stick with gini. So you should be good to go with library_w19_week3.py.
</div>